# Lab-10-2 mnist cnn

In [73]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

In [74]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [75]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [76]:
# MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [77]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [78]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__() 
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
            
            
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3*3*128, 625) 
        self.relu = nn.ReLU()        
        self.fc2 = nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [84]:
model = CNN().to(device)

In [85]:
model
value = torch.Tensor(1,1,28,28).to(device)
print( (model(value)) )

tensor([[ 0.0688,  0.1490,  0.0813, -0.1465,  0.1694, -0.0110,  0.0267,  0.1371,
         -0.2089,  0.2770]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [86]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [87]:
# training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost +=cost / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished')

[Epoch: 1] cost = 0.1680058389902115
[Epoch: 2] cost = 0.04433991387486458
[Epoch: 3] cost = 0.029901042580604553
[Epoch: 4] cost = 0.024756664410233498
[Epoch: 5] cost = 0.017989616841077805
[Epoch: 6] cost = 0.0162055641412735
[Epoch: 7] cost = 0.013459904119372368
[Epoch: 8] cost = 0.010287260636687279
[Epoch: 9] cost = 0.010298153385519981
[Epoch: 10] cost = 0.009686402976512909
[Epoch: 11] cost = 0.0074090249836444855
[Epoch: 12] cost = 0.006660947576165199
[Epoch: 13] cost = 0.006163207348436117
[Epoch: 14] cost = 0.007204665336757898
[Epoch: 15] cost = 0.006735054310411215
Learning Finished


In [89]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9884999990463257
